# `nn.module`

使用类似 PyTorch 的 API 创建、编译和运行神经网络的示例代码。

In [1]:
import tvm
from tvm.relay import Call
from tvm import relax, tir
from tvm.relax.testing import nn
from tvm.script import relax as R
import numpy as np

In [2]:
builder = relax.BlockBuilder()
n = tir.Var("n", "int64") # 符号变量用于表示小批量大小。
input_size = 784
hidden_sizes = [128, 32]
output_size = 10

构建用于分类任务的三层线性神经网络。

In [3]:
with builder.function("main"):
    model = nn.Sequential(
        nn.Linear(input_size, hidden_sizes[0]),
        nn.ReLU(),
        nn.Linear(hidden_sizes[0], hidden_sizes[1]),
        nn.ReLU(),
        nn.Linear(hidden_sizes[1], output_size),
        nn.LogSoftmax(),
    )
    data = nn.Placeholder((n, input_size), name="data")
    output = model(data)
    params = [data] + model.parameters()
    builder.emit_func_output(output, params=params)

获取并打印正在构建的 IRmodule。

In [4]:
mod = builder.get()
mod.show()

构建 IRModule 并创建 relax vm。

In [5]:
target = tvm.target.Target("llvm", host="llvm")
ex = relax.build(mod, target)
vm = relax.VirtualMachine(ex, tvm.cpu())

在 relax vm 上运行模型，输入数据的小批量大小为 3。

In [6]:
params = nn.init_params(mod) # 初始化参数
data = tvm.nd.array(np.random.rand(3, input_size).astype(np.float32))
res = vm["main"](data, *params)
print(res)

[[-2.3025851 -2.3025851 -2.3025851 -2.3025851 -2.3025851 -2.3025851
  -2.3025851 -2.3025851 -2.3025851 -2.3025851]
 [-2.3025851 -2.3025851 -2.3025851 -2.3025851 -2.3025851 -2.3025851
  -2.3025851 -2.3025851 -2.3025851 -2.3025851]
 [-2.3025851 -2.3025851 -2.3025851 -2.3025851 -2.3025851 -2.3025851
  -2.3025851 -2.3025851 -2.3025851 -2.3025851]]
